In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
import requests

In [87]:
import requests
import os
import logging
import pandas as pd
from dotenv import load_dotenv

class PySimFin:
    def __init__(self):
        self.url = "https://backend.simfin.com/api/v3/" 
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")

    def __load_dotenv(self):
        load_dotenv()

    def __create_headers(self):
        return {
            "Authorization": f"api-key {self.__token}",
            "Accept": "application/json"
        }

    def create_params_dictionary(self, ticker, start=None, end=None):
        params = {"ticker": ticker} 
        if start:
            params["start"] = start 
        if end:
            params["end"] = end 
        return params

    def get_share_prices(self, ticker: str, start: str = None, end: str = None):
        url = self.url + "companies/prices/compact"
        headers = self.__create_headers()
        params = self.create_params_dictionary(ticker, start, end)
    
        logging.info(f"Requesting: {url} with params: {params}")
    
        response = requests.get(url, headers=headers, params=params)
        
        print(f"Full URL: {response.url}")  
        print(f"Response Status: {response.status_code}")
    
        response.raise_for_status()
    
        data = response.json()
        if isinstance(data, list) and len(data) > 0 and "data" in data[0] and "columns" in data[0]:
            df = pd.DataFrame(data[0]["data"], columns=data[0]["columns"])
            return df
        else:
            logging.warning(f"No data found or format is unexpected for {ticker}")
            return pd.DataFrame()

            
    def get_financial_statement(self, ticker: str, statements: str, period: str= "FY" , start: str = None, end: str = None):
        url = self.url + "companies/statements/compact"
        headers = self.__create_headers()
        params = {
            "ticker": ticker,
            "statements": statements,
            "period": period
        }      
        
        if start:
            params["start"] = start 
        if end:
            params["end"] = end  
    
        logging.info(f"Requesting Financial Statement: {url} with params: {params}")
    
        try:
            response = requests.get(url, headers=headers, params=params)
            print(f"Full URL: {response.url}")
            print(f"Response Status: {response.status_code}")
            response.raise_for_status()
    
            data = response.json()
            if isinstance(data, list) and "statements" in data[0]:
                statement = data[0]["statements"][0]
                return pd.DataFrame(statement["data"], columns=statement["columns"])
            else:
                logging.warning(f"No financial statement data found for {ticker}.")
                return pd.DataFrame()
        except requests.exceptions.RequestException as e:
            logging.error(f"Error fetching financial statements: {e}")
            return pd.DataFrame()

    def get_general_data(self, ticker: str):
        url = self.url + "companies/general/compact"
        headers = self.__create_headers()
        params = {
            "ticker": ticker,
        }      
        
        logging.info(f"Requesting Financial Statement: {url} with params: {params}")
    
        try:
            response = requests.get(url, headers=headers, params=params)
            print(f"Full URL: {response.url}")
            print(f"Response Status: {response.status_code}")
            response.raise_for_status()
    
            data = response.json()
            if len(data["data"])>0:
                #general_data = data["data"][0]
                return pd.DataFrame(data["data"], columns=data["columns"])
            else:
                logging.warning(f"No financial statement data found for {ticker}.")
                return pd.DataFrame()       
        except requests.exceptions.RequestException as e:
            logging.error(f"Error fetching financial statements: {e}")
            return pd.DataFrame()



# Example usage
if __name__ == "__main__":
    simfin_api = PySimFin()

    df_prices = simfin_api.get_share_prices("AAPL", "2024-12-01", "2024-12-31") 
    #print(df_prices)

    df_financials = simfin_api.get_financial_statement("AAPL","Derived","FY")
    #print(df_financials.head())

    df_general=simfin_api.get_general_data("AAPL")



Full URL: https://backend.simfin.com/api/v3/companies/prices/compact?ticker=AAPL&start=2024-12-01&end=2024-12-31
Response Status: 200
Full URL: https://backend.simfin.com/api/v3/companies/statements/compact?ticker=AAPL&statements=Derived&period=FY
Response Status: 200
Full URL: https://backend.simfin.com/api/v3/companies/general/compact?ticker=AAPL
Response Status: 200


In [81]:
df_general

,companyDescription,endFy,id,industryName,isin,market,name,numEmployees,sectorCode,sectorName,ticker
0,Apple Inc is an American multinational technol...,9,111052,Technology,US0378331005,US,APPLE INC,147000,101001,Computer Hardware,AAPL


In [37]:
df_financials

,Fiscal Period,Fiscal Year,Report Date,TTM,Restated,Data Model,Gross Profit Margin,Operating Margin,EBITDA,"Earnings Per Share, Basic",...,Net Income (Adjusted),Net Profit Margin (Adjusted),Return on Equity (Adjusted),Return on Assets (Adjusted),Free Cash Flow to Net Income (Adjusted),Return On Invested Capital (Adjusted),Debt Ratio,Total Debt,Return on Equity,Return on Assets
0,FY,2018,2018-09-30,0,1,0.0,0.38344,0.26694,81801000000,3.00335,...,59531000000,0.22414,0.55560,0.16278,1.95041,0.27072,0.31295,114453883510,0.55560,0.16278
1,FY,2019,2019-09-30,0,1,0.0,0.37818,0.24572,76477000000,2.99145,...,55256000000,0.21238,0.61064,0.16323,1.32067,0.27857,0.31904,107999234571,0.61064,0.16323
2,FY,2020,2020-09-30,0,1,0.0,0.38233,0.24147,77344000000,3.30859,...,57411000000,0.20914,0.87866,0.17726,1.48761,0.30780,0.37750,122267418045,0.87866,0.17726
3,FY,2021,2021-09-30,0,1,0.0,0.41779,0.29782,120233000000,5.77609,...,94680000000,0.25882,1.50071,0.26974,1.25437,0.45264,0.38938,136673995139,1.50071,0.26974
4,FY,2022,2022-09-30,0,1,0.0,0.43310,0.30289,130541000000,6.15461,...,99803000000,0.25310,1.96959,0.28292,1.40610,0.53702,0.37574,132544717616,1.75407,0.28292
5,FY,2023,2023-09-30,0,1,0.0,0.44131,0.29821,125820000000,6.16067,...,96995000000,0.25306,1.56076,0.27510,1.28898,0.52283,0.31472,110965541724,1.72269,0.27510
6,FY,2024,2024-09-30,0,1,0.0,0.46206,0.31510,134661000000,6.10905,...,93736000000,0.23971,1.64594,0.25683,1.69214,0.59561,0.29190,106537713808,1.64594,0.25683


In [29]:
df_financials.columns

Index(['Fiscal Period', 'Fiscal Year', 'Report Date', 'Publish Date',
       'Restated', 'Source', 'TTM', 'Value Check', 'Data Model',
       'Cash, Cash Equivalents & Short Term Investments',
       'Cash & Cash Equivalents', 'Short Term Investments',
       'Accounts & Notes Receivable', 'Accounts Receivable, Net',
       'Notes Receivable, Net', 'Unbilled Revenues', 'Inventories',
       'Raw Materials', 'Work In Process', 'Finished Goods', 'Other Inventory',
       'Other Short Term Assets', 'Prepaid Expenses',
       'Derivative & Hedging Assets (Short Term)', 'Assets Held-for-Sale',
       'Deferred Tax Assets (Short Term)', 'Income Taxes Receivable',
       'Discontinued Operations (Short Term)',
       'Miscellaneous Short Term Assets', 'Total Current Assets',
       'Property, Plant & Equipment, Net', 'Property, Plant & Equipment',
       'Accumulated Depreciation', 'Long Term Investments & Receivables',
       'Long Term Investments', 'Long Term Marketable Securities',
      

In [15]:
df_prices

,Date,Dividend Paid,Common Shares Outstanding,Last Closing Price,Adjusted Closing Price,Highest Price,Lowest Price,Opening Price,Trading Volume
0,2024-12-02,None,15115823000,239.59,239.33,240.79,237.16,237.27,48137103
1,2024-12-03,None,15115823000,242.65,242.38,242.76,238.90,239.81,38861017
2,2024-12-04,None,15115823000,243.01,242.74,244.11,241.25,242.87,44383935
3,2024-12-05,None,15115823000,243.04,242.77,244.54,242.13,243.99,40033878
4,2024-12-06,None,15115823000,242.84,242.57,244.63,242.08,242.91,36870619
5,2024-12-09,None,15115823000,246.75,246.48,247.24,241.75,241.83,44649232
6,2024-12-10,None,15115823000,247.77,247.50,248.21,245.34,246.89,36914806
7,2024-12-11,None,15115823000,246.49,246.22,250.80,246.26,247.96,45205814
8,2024-12-12,None,15115823000,247.96,247.69,248.74,245.68,246.89,32777532
9,2024-12-13,None,15115823000,248.13,247.86,249.29,246.24,247.81,33155290
